In [1]:
# @title LIBRARY
!pip install pandas
!pip install nympy
!pip install tqdm
!pip install torch
!pip install torchvision
!pip install matplotlib
!pip install scikit-learn
!pip install torchsummary
!pip install tqdm

ERROR: Could not find a version that satisfies the requirement nympy (from versions: none)
ERROR: No matching distribution found for nympy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  A

In [12]:
# @title IMPORT_LIBRARY
import torch
from torch.utils.data import TensorDataset # Import TensorDataset
from torchvision.datasets  import CIFAR10
import torch.nn as nn
from torchsummary import  summary
from torchvision.transforms  import Compose , Resize,ToTensor
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import  DataLoader
from tqdm import  tqdm  #Tạo thanh Progress Bar
from sklearn.metrics import accuracy_score

# @title EfficientNet-B0 MODEL
import torch
import torch.nn as nn

# Swish Activation
class Swish(nn.Module):
    def forward(self, x):
        result = x * torch.sigmoid(x)
        if torch.isnan(result).any():
            raise ValueError("Gặp phải giá trị NaN trong kích hoạt Swish.")
        return result

# SE Block (Squeeze and Excitation)
class SE_Block(nn.Module):
    def __init__(self, in_channels, reduction=4):
        super(SE_Block, self).__init__()
        hidden_dim = in_channels // reduction
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),  # (C, H, W) -> (C, 1, 1)
            nn.Conv2d(in_channels, hidden_dim, 1),
            Swish(),
            nn.Conv2d(hidden_dim, in_channels, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.se(x)

# MBConv Block
class MBConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, expand_ratio, se_ratio=0.25):
        super(MBConv, self).__init__()
        hidden_dim = int(in_channels * expand_ratio)
        self.has_residual = (stride == 1 and in_channels == out_channels)
        self.has_se = se_ratio is not None and se_ratio > 0
        self.expand_ratio = expand_ratio

        # Expansion layer
        self.expand = nn.Sequential(
            nn.Conv2d(in_channels, hidden_dim, 1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            Swish()
        ) if expand_ratio != 1 else nn.Identity()

        # Depthwise convolution layer
        self.depthwise = nn.Sequential(
            nn.Conv2d(hidden_dim, hidden_dim, kernel_size=kernel_size, stride=stride, padding=kernel_size//2, groups=hidden_dim, bias=False),
            nn.BatchNorm2d(hidden_dim),
            Swish()
        )

        # Squeeze-and-Excitation block
        if self.has_se:
            self.se = SE_Block(hidden_dim, reduction=int(1 // se_ratio))

        # Project layer
        self.project = nn.Sequential(
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels)
        )

    def forward(self, x):
        out = self.expand(x)
        out = self.depthwise(out)
        if self.has_se:
            out = self.se(out)
        out = self.project(out)
        if self.has_residual:
            out = out + x  # Skip connection
        return out


# EfficientNet-B0
class My_EfficientNet_B0(nn.Module):
    def __init__(self, num_classes):
        super(My_EfficientNet_B0, self).__init__()
        base_channels = 32
        self.steam = nn.Sequential(
            nn.Conv2d(3, base_channels, 3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(base_channels),
            Swish()
        )

        in_channels = base_channels
        layers = []

        config = [
            (1, 16, 1, 3, 1),
            (6, 24, 2, 3, 2),
            (6, 40, 2, 5, 2),
            (6, 80, 3, 3, 2),
            (6, 112, 3, 5, 1),
            (6, 192, 4, 5, 2),
            (6, 320, 1, 3, 1)
        ]

        # Chỉnh sửa cấu hình để đảm bảo sự phù hợp về kênh
        for expand_ratio, out_channels, num_layers, kernel_size, stride in config:
            for i in range(num_layers):
                s = stride if i == 0 else 1
                layers.append(MBConv(in_channels, out_channels, kernel_size, s, expand_ratio))
                in_channels = out_channels

        self.body = nn.Sequential(*layers)

        self.head = nn.Sequential(
            nn.Conv2d(in_channels, 1280, 1, bias=False),
            nn.BatchNorm2d(1280),
            Swish(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(1280, num_classes)
        )

    def forward(self, x):
        x = self.steam(x)
        x = self.body(x)
        x = self.head(x)
        return x




In [10]:


# @title IMPORT_LIBRARY
import torch
from torch.utils.data import TensorDataset # Import TensorDataset
from torchvision.datasets  import CIFAR10
import torch.nn as nn
from torchsummary import  summary
from torchvision.transforms  import Compose , Resize,ToTensor
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import  DataLoader
from tqdm import  tqdm  #Tạo thanh Progress Bar
from sklearn.metrics import accuracy_score


# @title MODEL RESNET 18
class BasicBlock(nn.Module):
  def __init__(self, in_channels,out_channels, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU(inplace=False)
    self.conv2 = nn.Conv2d(out_channels,out_channels,kernel_size=3, stride=1,padding=1,bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.shortcut = nn.Sequential()
    if stride != 1 or in_channels != out_channels:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_channels,out_channels, kernel_size=1, stride=stride,bias = False),
          nn.BatchNorm2d(out_channels)

      )
  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)  # ReLU should not modify the tensor in-place
    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)  # Again, no in-place operation

    out = out + self.shortcut(x)
    out = self.relu(out)
    return out


class Resnet18(nn.Module):
  def __init__(self, num_classes = 10):
    super(Resnet18,self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(in_channels=3,out_channels=64, kernel_size=3, stride=1,padding=1, bias=False) #Giu nguyen size vi Cfar size be
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace=False)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.layer1 = self.make_layer(64,BasicBlock, 2,   stride=1)
    self.layer2 = self.make_layer(128,BasicBlock, 2,  stride=2)
    self.layer3 = self.make_layer(256,BasicBlock, 2,  stride=2)
    self.layer4 = self.make_layer(512,BasicBlock, 2,  stride=2)

    self.avg = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512, num_classes)


  def make_layer(self, out_channels,block, num_block, stride):
    strides = [stride] + [1]*(num_block-1)
    layer = []
    for stride in strides:
      layer.append(block(self.in_channels,out_channels, stride))
      self.in_channels = out_channels

    return nn.Sequential(*layer)

  def forward(self,x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.maxpool(out)

    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)

    out = self.avg(out)
    out = out.view(out.size(0),-1)
    out = self.fc(out)
    return out



In [15]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
import sys
sys.path.append('/kaggle/input/dataclassification')
 
from Dataset_Classification import Falling_Dataset4Clss
import torch
import os
from sklearn.metrics import accuracy_score
from torch.utils.data import  DataLoader
from torchvision.transforms  import Compose , Resize,ToTensor,RandomAffine, ColorJitter,Normalize
import  torch.nn as nn
from tqdm import  tqdm  #Tạo thanh Progress Bar
from torch.utils.tensorboard import SummaryWriter
def compute_and_save_soft_labels(model_teacher, train_dataset, save_path, batch_size=64, T=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_teacher.to(device)
    model_teacher.eval()

    dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        drop_last=False,
        pin_memory=True
    )

    all_soft_labels = []

    with torch.no_grad():
        for images, _ in tqdm(dataloader, desc="Computing soft labels in memory"):
            images = images.to(device, non_blocking=True)
            outputs = model_teacher(images)
            soft = torch.softmax(outputs / T, dim=1)
            all_soft_labels.append(soft.cpu())
            torch.cuda.empty_cache()


    soft_labels_tensor = torch.cat(all_soft_labels, dim=0)
    torch.save(soft_labels_tensor, save_path)
    print(f"Saved soft labels to: {save_path}")


# --------- STEP 2: Train student với soft labels đã lưu ----------
def train_with_KD_soft_labels(model_student, train_dataset, test_dataloader, soft_labels_path,
                              check_point, num_epochs, pathsave_model, model_name, tensorboard_name, T=2):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_student = model_student.to(device)
    writer = SummaryWriter(log_dir=pathsave_model, comment=model_name, filename_suffix=tensorboard_name)

    optimizer = torch.optim.Adam(model_student.parameters(), lr=1e-3, betas=(0.9, 0.999))
    criterion = nn.CrossEntropyLoss()

    if check_point is not None:
        checkpoint = torch.load(check_point)
        start_epoch = checkpoint['epoch']
        model_student.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
    else:
        start_epoch = 0

    soft_labels = torch.load(soft_labels_path)  # (N, num_classes)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=2,
        drop_last=False,
        pin_memory=True
    )

    best_accuracy = 0
    num_iters = len(train_loader)
    for epoch in range(start_epoch, num_epochs):
        model_student.train()
        progress_bar =  tqdm(train_loader)

        for i, (images, labels) in enumerate(progress_bar):
            images = images.to(device)
            labels = labels.to(device)
            soft_targets = soft_labels[i * 64:(i + 1) * 64].to(device)

            outputs = model_student(images)

            ce_loss = criterion(outputs, labels)
            kd_loss = nn.KLDivLoss(reduction='batchmean')(
                torch.log_softmax(outputs / T, dim=1), soft_targets
            )
            loss = 0.85 * ce_loss + 0.15 * (T ** 2) * kd_loss
            progress_bar.set_description('Epoch {}/{}, Iteration {}/{}, Loss {:.3f}'.format( epoch+1, num_epochs, i+1, num_iters, loss))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            writer.add_scalar('Train/Loss', loss.item(), epoch * len(train_loader) + i)

        # Validation
        model_student.eval()
        all_preds, all_gts = [], []

        with torch.no_grad():
            for images, labels in test_dataloader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model_student(images)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_gts.extend(labels.cpu().numpy())

        acc = accuracy_score(all_gts, all_preds)
        writer.add_scalar('Val/Accuracy', acc, epoch)

        # Save model
        checkpoint = {
            'epoch': epoch + 1,
            'model': model_student.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        torch.save(checkpoint, f'{pathsave_model}/last_cnn.pt')

        if acc > best_accuracy:
            torch.save(checkpoint, f'{pathsave_model}/best_cnn.pt')
            best_accuracy = acc

        print(f"Epoch {epoch + 1} | Accuracy: {acc:.4f} | Best: {best_accuracy:.4f}")
from torchvision import models
def make_dataset(type_data):
  if type_data == 'train':
    train_transform = Compose([
          # Biến đổi hình học
          RandomAffine(
              degrees=(-5, 5),
              translate=(0.05, 0.05),
              scale=(0.85, 1.15),  # scale quanh 1, tránh thu nhỏ quá nhiều
              shear=5              # shear nhẹ
          ),
          # Biến đổi màu sắc
          ColorJitter(
              brightness=0.1,
              contrast=0.2,
              saturation=0.2,
              hue=0.05
          ),
          Resize((224, 224)),
          ToTensor(),
          # Chuẩn hóa theo ImageNet
          Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]
          )
    ])
  else:
    train_transform = Compose([
          Resize((224, 224)),
          ToTensor(),
          # Chuẩn hóa theo ImageNet
          Normalize(
              mean=[0.485, 0.456, 0.406],
              std=[0.229, 0.224, 0.225]
          )
    ])


  dataset = Falling_Dataset4Clss(
      root='/kaggle/input/classification-dataset',
      type_data = type_data,
      transform = train_transform
  )

  return dataset

if __name__ == '__main__':
    import torch
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    model_vit = models.vit_b_16(pretrained=False)
    model_vit.heads.head = nn.Linear(model_vit.heads.head.in_features, 2)

    # 2. Load checkpoint và trích xuất state_dict
    checkpoint = torch.load('/kaggle/input/vit-hehe/best_cnn.pt', map_location='cpu')
    model_vit.load_state_dict(checkpoint['model'])
    train_dataset = make_dataset('train')
    # 4. Đưa model sang thiết bị và chuyển sang eval mode
    model_vit = model_vit.to(device)
    model_vit.eval()
    # compute_and_save_soft_labels(model_vit, train_dataset,
    #                              'D:\BTL_ThucTapCS/soft_labels.pt', batch_size=64,
    #                              T=2)

    test_dataset = make_dataset('valid')
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=64,
        num_workers=2,
        drop_last=False,
        pin_memory=True

    )
    # Load checkpoint vào CPU
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    # Load vào mô hình
    model_res = Resnet18(num_classes = 2)
    checkpoint = torch.load('/kaggle/input/resnet-ahihi/best_cnn.pt', map_location='cpu')
    model_res.load_state_dict(checkpoint['model'])
    
    # Chỉ chuyển sang GPU nếu còn đủ bộ nhớ
    model_res.to(device)

    train_with_KD_soft_labels(
        model_student = model_res,
        train_dataset = train_dataset,
        test_dataloader = test_dataloader,
        soft_labels_path ='/kaggle/input/soft-labels/soft_labels.pt',
        check_point = None,
        num_epochs = 30,
        pathsave_model = '/kaggle/working/',
        model_name = 'res_withKD',
        tensorboard_name= 'run1',
        T=2
    )

/tmp/ipykernel_31/2583731046.py:189: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/kaggle/input/vit-hehe/best_cnn.pt', map_location='cpu')
/tmp/ipy

KeyboardInterrupt: 